In [14]:
import importlib
spec = importlib.util.find_spec("pandastable")
if spec is None:
    %pip install pandastable
import tkinter as tk
import sys
from tkinter import ttk,messagebox,font
from pandastable import Table
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

#general defines
dtype_dict = {
    'YEAR': int,
    'MONTH': int,
    'SUPPLIER': str,
    'ITEM CODE': str,
    'ITEM DESCRIPTION': str,
    'ITEM TYPE': str,
    'RETAIL SALES': float,
    'RETAIL TRANSFERS': float,
    'WAREHOUSE SALES': float}

window = tk.Tk()
style = ttk.Style()
screen_width = window.winfo_screenwidth()
screen_height = window.winfo_screenheight()
def validate_login(username, password):
    # Adding validation logic here
    if username == "admin" and password == "123":
        messagebox.showinfo("Login info", "Welcome Admin!")
        window.destroy()
        return True
    else:
        messagebox.showinfo("Login info", "Incorrect credentials")
        window.destroy()
        sys.exit()
        return False
     

def resize(event):
    # Calculate new font size based on window size
    new_size = max(8, min(int(event.width / 50), int(event.height / 30)))
    style.configure("TButton", font=("Rockwell", new_size))
    style.configure("TEntry", font=("Rockwell", new_size))

window.bind('<Configure>', resize)   

def create_login_window():
    window.title("Login Screen")
    window.geometry('800x600') 
    window.resizable(True, True)
    window_height = 800
    window_width = 600
    screen_width = window.winfo_screenwidth()
    screen_height = window.winfo_screenheight()
    position_top = int(screen_height // 2 - window_height // 2)
    position_right = int(screen_width // 2 - window_width // 2)
    # background color
    window.configure(bg='#61A0AF')

    # style creation
    style = ttk.Style()
    style.configure("TEntry",
                    foreground="#F06C9B",
                    fieldbackground="#F06C9B",
                    bordercolor="#F06C9B",
                    lightcolor="#F06C9B",
                    darkcolor="#F06C9B",
                    borderwidth=20,
                    relief="groove",
                    font=("Rockwell", 14))

    style.configure("TButton",
                    background="#F5D491",
                    foreground="black",
                    borderwidth=20,
                    relief="groove",
                    font=("Rockwell", 14))

    username_label = ttk.Label(window, text="Username", font=("Rockwell", 14), background='#61A0AF', foreground="blue",anchor='center', justify='center')
    username_label.pack(fill=tk.BOTH, expand=1)
    username_entry = ttk.Entry(window,style='TEntry')
    username_entry.pack(fill=tk.BOTH, expand=1)

    password_label = ttk.Label(window, text="Password",font=("Rockwell", 14), background='#61A0AF', foreground="blue",anchor='center', justify='center')
    password_label.pack(fill=tk.BOTH, expand=1)
    password_entry = ttk.Entry(window,style="TEntry",show="*")
    password_entry.pack(fill=tk.BOTH, expand=1)

    submit_button = ttk.Button(window, text="Login",style="TButton", command=lambda: validate_login(username_entry.get(), password_entry.get()))
    submit_button.pack(fill=tk.BOTH, expand=1)
    window.mainloop()

#Creating loading window

def create_loading_window():
    loading_window = tk.Toplevel()
    loading_window.title("Loading...")
    screen_width = loading_window.winfo_screenwidth()
    screen_height = loading_window.winfo_screenheight()
    loading_window.geometry('300x100') 
    window_height = 300
    window_width = 100
    loading_window.resizable(False, False)
    position_top = int(screen_height // 2 - window_height // 2)
    position_right = int(screen_width // 2 - window_width // 2)

    # Add a label with a custom font and color
    loading_label = ttk.Label(loading_window, text="Loading data, please wait...", font=("Helvetica", 16), foreground="blue")
    loading_label.pack(pady=10)

    # Add a progress bar
    progress = ttk.Progressbar(loading_window, length=200, mode='determinate')
    progress.pack(pady=10)

    return loading_window, progress

# loading data in chunks and reporting progress

def load_data(filename, loading_window, progress):
    chunksize = 10000
    chunks = []
    total_rows = sum(1 for row in open(filename, 'r')) - 1 # no of rows - header 

    for i, chunk in enumerate(pd.read_csv(filename, chunksize=chunksize)):
        chunks.append(chunk)

        #update progress bar
        progress['value'] = (i+1) * chunksize / total_rows * 100
        loading_window.update()
    # concatenate all chunks into one dataframe
    data = pd.concat(chunks, axis=0)
    data = data.dropna()
    data['DATE']= pd.to_datetime(data[['YEAR','MONTH']].assign(DAY=1))
    data = data.sort_values(by='DATE')
    # delete loading window
    loading_window.destroy()

    return(data)

def treeview_sort_column(tv, col, reverse):
    l = [(tv.set(k, col), k) for k in tv.get_children('')]
    l.sort(reverse=reverse)

    #rearrange items in sorted position
    for index, (val, k) in enumerate(l):
        tv.move(k,'', index)
    # reverse sort next time
    tv.heading(col, command=lambda: treeview_sort_column(tv, col, not reverse))
#paging defs
items_per_page = 10000
current_page = 0

def display_page(page):
    # Clear the treeview
    for i in tree.get_children():
        tree.delete(i)

    # Calculate the range of items to display
    start = page * items_per_page
    end = start + items_per_page

    # Insert the items for this page into the treeview
    for item in items[start:end]:
        tree.insert('', 'end', values=item)

def next_page():
    global current_page
    current_page += 1
    display_page(current_page)

def prev_page():
    global current_page
    if current_page > 0:
        current_page -= 1
        display_page(current_page)

def create_data_window():
    #create progress bar 
    loading_window, progress = create_loading_window()
    # Load the data
    data = load_data('Warehouse_and_Retail_Sales.csv', loading_window, progress)

    # Group the data by year and sum the 'RETAIL SALES' for each year
    yearly_sales = data.groupby(data['DATE'].dt.year)['RETAIL SALES'].sum()

    # Create a new tkinter window
    window = tk.Toplevel()
    window.title("Data Window")
    screen_width = window.winfo_screenwidth()
    screen_height = window.winfo_screenheight()
    window_width = int(screen_width * 0.8)
    window_height = int(screen_height * 0.8)
    position_top = int(screen_height // 2 - window_height // 2)
    position_right = int(screen_width // 2 - window_width // 2)
    window.geometry(f"{window_width}x{window_height}+{position_right}+{position_top}")
    # Create a Frame for the Treeview
    tree_frame = tk.Frame(window)
    tree_frame.grid(row=1,column=0)

    # create a scrollbar
    scrollbar = ttk.Scrollbar(tree_frame)
    scrollbar.pack(side='left', fill='y')

    #Create a Treeview widget and display the data in it
    global tree
    tree = ttk.Treeview(tree_frame, columns=list(data.columns), show = 'headings', yscrollcommand=scrollbar.set)
    for column in data.columns:
            tree.heading(column, text=column, command=lambda _col=column: treeview_sort_column(tree, _col, False))
    # Create a list of items
    global items
    items = [tuple(x) for x in data.values]

    # Display the first page of items
    display_page(current_page)

    tree.pack(side='left', fill='both', expand=True)
    scrollbar.config(command=tree.yview)
    # Add buttons to go to the next and previous pages
    prev_button = ttk.Button(window, text="Previous Page", command=prev_page)
    prev_button.grid(row=2, column=0, sticky='w')

    next_button = ttk.Button(window, text="Next Page", command=next_page)
    next_button.grid(row=2, column=0, sticky='nw')

    
    # Create a Frame for the chart
    chart_frame = tk.Frame(window)
    chart_frame.grid(row=2, column=1)
    # Group the data by 'DATE' and sum the 'RETAIL SALES' for each date
    sales_data = data.groupby('DATE')['RETAIL SALES'].sum().reset_index()
    # Create a chart and display the sales data in it
    fig = plt.Figure(figsize=(5, 4), dpi=100)
    ax = fig.add_subplot(111)
    ax.plot(data['DATE'], data['RETAIL SALES'])
    chart = FigureCanvasTkAgg(fig, master=chart_frame)
    chart.draw()
    chart.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

    # Create a Frame for the detailed analysis
    analysis_frame = tk.Frame(window)
    analysis_frame.grid(row=0,column=1)

    # Create a Text widget and display the detailed analysis in it
    analysis = tk.Text(analysis_frame, height=20, width=30)
    analysis.pack()
    analysis.insert(tk.END, "Here is the detailed analysis:\n\n")
    analysis.insert(tk.END, "Total sales: " + str(data['RETAIL SALES'].sum()) + "\n")
    analysis.insert(tk.END, "Average monthly sales: " + str(data['RETAIL SALES'].mean()) + "\n")
    # ... add more analysis here ...

    window.mainloop()

def main():
    login_window = create_login_window()
    if create_login_window:
        create_data_window()
    


if __name__ == "__main__":
        main()